# Discover & Process Sentinel-1 InSAR pairs

- The provider used for this notebook is `cop_dataspace`
- A free account needs to be created at https://dataspace.copernicus.eu/
- The credentials need to be configured as explained here https://eodag.readthedocs.io/en/stable/getting_started_guide/configure.html

In [ ]:
%load_ext autoreload
%autoreload 2

from eo_tools.util import explore_products
from eo_tools.util import visualize_insar_phase, visualize_insar_coh, visualize_sar_intensity
from eo_tools.S1 import process_InSAR
import geopandas as gpd
import folium
import folium.plugins
import logging

from eodag import EODataAccessGateway
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

## Select an area of interest

- Select a drawing tool (polygon, circle or rectangle)
- Draw the AOI on the map
- Click "Export" on the right to save the GeoJSON file

In [ ]:
map_select = folium.Map()
folium.plugins.Draw(
    export=True,
    draw_options={
        "polyline": False,
        "circlemarker": False,
        "marker": False,
    },
).add_to(map_select)
map_select

## Search

In [ ]:
# load a geometry
file_aoi = "../data/Morocco_AOI.geojson"
shp = gpd.read_file(file_aoi).geometry[0]
search_criteria = {
    "productType": "S1_SAR_SLC",
    "start": "2023-09-01",
    "end": "2023-09-20",
    "geom": shp
}
results, _ = dag.search(**search_criteria)

## Visualize by groups

- For InSAR, products must be overlapping and from the same relative orbit.
- They also have to be of the same ascending / descending configuration.

In [ ]:
# TODO: add filter on ascending / descending
explore_products(results, shp)

## Download

In [ ]:
# fill indices of products to download
indices_to_download = [2, 3]
to_download = [results[i] for i in indices_to_download]

# if the data has already been downloaded, just re-run the cell to get the product paths
dl = dag.download_all(to_download, directory_path="/data/S1/", extract=False)

# Process

In [ ]:
# extract paths for processing
process_InSAR(
    dl[0],
    dl[1],
    out_dir="/data/res",
    tmp_dir="/data/tmp",
    shp=shp,
    pol="VV",
    coh_only=False,
    intensity=True,
    clear_tmp_files=True,
    resume=False,
)

In [ ]:
file_phase = "/data/res/phi_VV_15Sep2023_03Sep2023_slice2_crop.tif"
visualize_insar_phase(file_phase)

In [ ]:
file_coh = "/data/res/coh_VV_15Sep2023_03Sep2023_slice2_crop.tif"
visualize_insar_coh(file_coh)

In [ ]:
file_int = "/data/res/int_VV_15Sep2023_slice2_crop.tif"
visualize_sar_intensity(file_int)